<a href="https://colab.research.google.com/github/OmegaTrees/JupiterLab-Render/blob/main/Unzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Install required packages
!pip install pyrogram tgcrypto rarfile py7zr nest_asyncio

import os
import zipfile
import rarfile
import py7zr
import shutil
import asyncio
import time
import re
import threading
from pyrogram import Client, filters
from pyrogram.types import Message
import nest_asyncio

# Fix for Colab's event loop
nest_asyncio.apply()

# Configuration - Using your API credentials for 2GB uploads
API_ID = 25U92
API_HASH = "8206UUJjj12a"
BOT_TOKEN = "7985393UuuuH3v97nfTY"

# File size limit (2GB for user accounts)
MAX_FILE_SIZE = 2 * 1024 * 1024 * 1024  # 2GB

print("🤖 Advanced Unzip Bot Started - All updates will show in Telegram!")

app = Client("unzip_bot", api_id=API_ID, api_hash=API_HASH, bot_token=BOT_TOKEN)

# Global progress tracking
progress_data = {}

def format_size(size):
    for unit in ['B', 'KB', 'MB', 'GB']:
        if size < 1024:
            return f"{size:.1f} {unit}"
        size /= 1024
    return f"{size:.1f} TB"

def format_time(seconds):
    if seconds < 60:
        return f"{seconds:.1f}s"
    elif seconds < 3600:
        return f"{seconds/60:.1f}m"
    else:
        return f"{seconds/3600:.1f}h"

def is_split_archive(filename):
    """Check if file is part of a split archive"""
    patterns = [
        r'\.part\d+\.rar$',  # .part1.rar, .part2.rar
        r'\.r\d+$',          # .r01, .r02
        r'\.\d+$',           # .001, .002
        r'\.z\d+$',          # .z01, .z02
        r'\.7z\.\d+$',       # .7z.001, .7z.002
    ]

    for pattern in patterns:
        if re.search(pattern, filename.lower()):
            return True
    return False

def is_archive(filename):
    """Check if file is an archive (including split archives)"""
    extensions = ['.zip', '.rar', '.7z', '.tar', '.gz', '.bz2']

    # Check regular extensions
    for ext in extensions:
        if filename.lower().endswith(ext):
            return True

    # Check split archive patterns
    return is_split_archive(filename)

def get_split_archive_parts(base_file, directory):
    """Find all parts of a split archive"""
    parts = []
    base_name = os.path.basename(base_file)

    # Different patterns for split archives
    patterns = [
        # For .001, .002, .003 format
        (r'(.+)\.(\d+)$', lambda m: f"{m.group(1)}.{int(m.group(2)):03d}"),
        # For .part1.rar, .part2.rar format
        (r'(.+)\.part(\d+)\.rar$', lambda m: f"{m.group(1)}.part{m.group(2)}.rar"),
        # For .r01, .r02 format
        (r'(.+)\.r(\d+)$', lambda m: f"{m.group(1)}.r{int(m.group(2)):02d}"),
        # For .7z.001, .7z.002 format
        (r'(.+\.7z)\.(\d+)$', lambda m: f"{m.group(1)}.{int(m.group(2)):03d}"),
    ]

    for pattern, formatter in patterns:
        match = re.match(pattern, base_name)
        if match:
            # Find all parts with this pattern
            part_num = 1
            while True:
                try:
                    if pattern == r'(.+)\.(\d+)$':
                        part_name = f"{match.group(1)}.{part_num:03d}"
                    elif pattern == r'(.+)\.part(\d+)\.rar$':
                        part_name = f"{match.group(1)}.part{part_num}.rar"
                    elif pattern == r'(.+)\.r(\d+)$':
                        part_name = f"{match.group(1)}.r{part_num:02d}"
                    elif pattern == r'(.+\.7z)\.(\d+)$':
                        part_name = f"{match.group(1)}.{part_num:03d}"

                    part_path = os.path.join(directory, part_name)
                    if os.path.exists(part_path):
                        parts.append(part_path)
                        part_num += 1
                    else:
                        break
                except:
                    break

            if parts:
                return sorted(parts)

    return [base_file] if os.path.exists(base_file) else []

def extract_file(file_path, extract_to):
    """Extract archive"""
    try:
        if file_path.lower().endswith('.zip'):
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)

        elif file_path.lower().endswith('.rar'):
            with rarfile.RarFile(file_path, 'r') as rar_ref:
                rar_ref.extractall(extract_to)

        elif file_path.lower().endswith('.7z'):
            with py7zr.SevenZipFile(file_path, mode='r') as sz_ref:
                sz_ref.extractall(extract_to)

        # Handle split archives (.001, .002, etc.)
        elif is_split_archive(file_path):
            # Try different extraction methods for split archives
            try:
                # First try as RAR
                with rarfile.RarFile(file_path, 'r') as rar_ref:
                    rar_ref.extractall(extract_to)
            except:
                try:
                    # Try as 7z
                    with py7zr.SevenZipFile(file_path, mode='r') as sz_ref:
                        sz_ref.extractall(extract_to)
                except:
                    # Try as zip
                    with zipfile.ZipFile(file_path, 'r') as zip_ref:
                        zip_ref.extractall(extract_to)
        else:
            return False

        return True

    except Exception as e:
        return False

def get_all_files(directory):
    """Get all files recursively"""
    files = []
    for root, dirs, filenames in os.walk(directory):
        for filename in filenames:
            files.append(os.path.join(root, filename))
    return files

async def progress_updater(message_id):
    """Background task to update progress messages"""
    while message_id in progress_data:
        try:
            data = progress_data[message_id]
            msg = data['message']
            text = data['text']

            if text:
                await msg.edit_text(text)

            await asyncio.sleep(3)  # Update every 3 seconds
        except:
            pass

@app.on_message(filters.command("start"))
async def start(client, message):
    await message.reply_text(
        "🤖 **Advanced Unzip Bot**\n\n"
        "✅ **Features:**\n"
        "• 2GB file upload support\n"
        "• Real-time progress in Telegram\n"
        "• Split archive support\n"
        "• Multiple format support\n\n"
        "📦 **Supported formats:**\n"
        "• ZIP (.zip)\n"
        "• RAR (.rar, .part1.rar, .r01)\n"
        "• 7Z (.7z, .7z.001)\n"
        "• Split archives (.001, .002, etc.)\n\n"
        "📱 **All progress updates will show here in Telegram!**\n\n"
        "Just send me your archive! 🚀"
    )

@app.on_message(filters.document)
async def handle_archive(client, message):
    doc = message.document
    file_name = doc.file_name
    file_size = doc.file_size

    # Send initial message to Telegram
    await message.reply_text(
        f"📥 **File Received**\n\n"
        f"📁 Name: {file_name}\n"
        f"📊 Size: {format_size(file_size)}\n"
        f"🔍 Checking file type..."
    )

    # Check if it's an archive (including split archives)
    if not is_archive(file_name):
        await message.reply_text(
            "❌ **Not a supported archive!**\n\n"
            "Supported formats:\n"
            "📦 ZIP, RAR, 7Z\n"
            "📦 Split archives (.001, .part1.rar, etc.)\n\n"
            "Send me an archive file!"
        )
        return

    # Archive detected message
    await message.reply_text(
        f"✅ **Archive Detected!**\n\n"
        f"📦 File: {file_name}\n"
        f"📊 Size: {format_size(file_size)}\n"
        f"🔍 Type: {'Split Archive' if is_split_archive(file_name) else 'Regular Archive'}\n\n"
        f"📥 **Starting download...**"
    )

    # Progress tracking
    progress_msg = await message.reply_text(
        f"📥 **Downloading...**\n\n"
        f"📁 {file_name}\n"
        f"📊 0% - 0 B/s\n"
        f"⏱️ Starting..."
    )

    # Set up progress tracking
    message_id = message.id
    progress_data[message_id] = {
        'message': progress_msg,
        'text': None
    }

    # Start progress updater
    updater_task = asyncio.create_task(progress_updater(message_id))

    try:
        # Download with progress
        download_path = f"/content/{file_name}"

        start_time = time.time()
        last_update = [0]  # Use list to modify from inner function

        def progress_callback(current, total):
            # Store progress data for background updater
            now = time.time()
            if now - last_update[0] > 2:  # Store update every 2 seconds
                last_update[0] = now

                elapsed = now - start_time
                percentage = (current / total * 100) if total > 0 else 0
                speed = current / elapsed if elapsed > 0 else 0
                eta = (total - current) / speed if speed > 0 else 0

                progress_bar = "█" * int(percentage / 10) + "░" * (10 - int(percentage / 10))

                progress_text = (
                    f"📥 **Downloading**\n\n"
                    f"📁 {file_name[:30]}{'...' if len(file_name) > 30 else ''}\n\n"
                    f"[{progress_bar}] {percentage:.1f}%\n\n"
                    f"📊 {format_size(current)} / {format_size(total)}\n"
                    f"⚡ Speed: {format_size(speed)}/s\n"
                    f"⏱️ ETA: {format_time(eta)}\n"
                    f"🕐 Elapsed: {format_time(elapsed)}"
                )

                # Update progress data for background task
                if message_id in progress_data:
                    progress_data[message_id]['text'] = progress_text

        # Download file
        await message.download(
            file_name=download_path,
            progress=progress_callback
        )

        download_time = time.time() - start_time
        avg_speed = file_size / download_time if download_time > 0 else 0

        # Stop progress updater
        updater_task.cancel()
        del progress_data[message_id]

        await progress_msg.edit_text(
            f"✅ **Download Complete!**\n\n"
            f"📁 File: {file_name}\n"
            f"📊 Size: {format_size(file_size)}\n"
            f"⚡ Avg Speed: {format_size(avg_speed)}/s\n"
            f"⏱️ Time: {format_time(download_time)}\n\n"
            f"🔍 **Checking for split parts...**"
        )

        # Check for split archive parts
        download_dir = "/content"
        all_parts = get_split_archive_parts(download_path, download_dir)

        if len(all_parts) > 1:
            await progress_msg.edit_text(
                f"🔗 **Split Archive Detected!**\n\n"
                f"📦 Found {len(all_parts)} parts\n"
                f"📁 Main file: {file_name}\n\n"
                f"✅ All parts available\n"
                f"🚀 Ready for extraction!"
            )
            extract_file_path = all_parts[0]  # Use first part for extraction
        else:
            await progress_msg.edit_text(
                f"📦 **Single Archive File**\n\n"
                f"📁 File: {file_name}\n"
                f"📊 Size: {format_size(file_size)}\n\n"
                f"🚀 Ready for extraction!"
            )
            extract_file_path = download_path

        await asyncio.sleep(2)

        # Extract
        await progress_msg.edit_text(
            f"📦 **Starting Extraction...**\n\n"
            f"📁 Extracting: {os.path.basename(extract_file_path)}\n"
            f"⏳ Please wait...\n\n"
            f"📊 This may take a while for large archives"
        )

        extract_dir = f"/content/extracted_{message.id}"
        os.makedirs(extract_dir, exist_ok=True)

        extract_start = time.time()

        # Send periodic extraction updates
        async def send_extraction_updates():
            while True:
                await asyncio.sleep(15)  # Update every 15 seconds
                elapsed = time.time() - extract_start
                try:
                    await progress_msg.edit_text(
                        f"📦 **Extracting Archive...**\n\n"
                        f"📁 File: {os.path.basename(extract_file_path)}\n"
                        f"⏳ Processing archive contents...\n"
                        f"🕐 Elapsed: {format_time(elapsed)}\n\n"
                        f"⏳ Please wait, extracting large archives takes time..."
                    )
                except:
                    break

        # Start extraction updates
        extraction_task = asyncio.create_task(send_extraction_updates())

        try:
            success = extract_file(extract_file_path, extract_dir)
        finally:
            extraction_task.cancel()

        extract_time = time.time() - extract_start

        if not success:
            await progress_msg.edit_text(
                "❌ **Extraction Failed!**\n\n"
                "Possible causes:\n"
                "• Corrupted archive\n"
                "• Missing split parts\n"
                "• Unsupported format\n"
                "• Password protected\n"
                "• Invalid archive structure"
            )
            return

        await progress_msg.edit_text(
            f"✅ **Extraction Complete!**\n\n"
            f"📦 Archive: {file_name}\n"
            f"⏱️ Extract time: {format_time(extract_time)}\n"
            f"🔍 Scanning extracted files..."
        )

        await asyncio.sleep(2)

        # Get extracted files
        files = get_all_files(extract_dir)
        if not files:
            await progress_msg.edit_text("❌ **No files found in archive!**")
            return

        # Calculate total size
        total_size = sum(os.path.getsize(f) for f in files)

        await progress_msg.edit_text(
            f"✅ **Files Ready for Upload!**\n\n"
            f"📦 Archive: {file_name}\n"
            f"📁 Files found: {len(files)}\n"
            f"📊 Total size: {format_size(total_size)}\n"
            f"⏱️ Extract time: {format_time(extract_time)}\n\n"
            f"🚀 **Starting upload process...**"
        )

        await asyncio.sleep(2)

        # Upload files with progress
        uploaded_size = 0
        success_count = 0
        upload_start = time.time()

        # Set up upload progress tracking
        upload_progress_data = {
            'message': progress_msg,
            'text': None
        }

        async def upload_progress_updater():
            while upload_progress_data['message']:
                try:
                    if upload_progress_data['text']:
                        await upload_progress_data['message'].edit_text(upload_progress_data['text'])
                    await asyncio.sleep(4)  # Update every 4 seconds
                except:
                    pass

        upload_updater_task = asyncio.create_task(upload_progress_updater())

        for i, file_path in enumerate(files, 1):
            try:
                file_size_current = os.path.getsize(file_path)
                base_name = os.path.basename(file_path)

                # Update overall progress
                elapsed = time.time() - upload_start
                avg_speed = uploaded_size / elapsed if elapsed > 0 else 0
                remaining_files = len(files) - i + 1
                eta = (remaining_files * (total_size - uploaded_size) / len(files) / avg_speed) if avg_speed > 0 else 0

                upload_progress_data['text'] = (
                    f"📤 **Uploading Files**\n\n"
                    f"📁 Current: {base_name[:25]}{'...' if len(base_name) > 25 else ''}\n"
                    f"📊 Progress: {i}/{len(files)} files\n"
                    f"📈 Completed: {format_size(uploaded_size)}/{format_size(total_size)}\n"
                    f"⚡ Avg Speed: {format_size(avg_speed)}/s\n"
                    f"⏱️ ETA: {format_time(eta)}\n"
                    f"🕐 Elapsed: {format_time(elapsed)}"
                )

                # Check file size limit
                if file_size_current > MAX_FILE_SIZE:
                    await message.reply_text(
                        f"⚠️ **File Too Large**\n\n"
                        f"📁 File: {base_name}\n"
                        f"📊 Size: {format_size(file_size_current)}\n"
                        f"📏 Limit: {format_size(MAX_FILE_SIZE)}\n"
                        f"❌ Skipping this file"
                    )
                    continue

                caption = (
                    f"📁 **{base_name}**\n"
                    f"📊 Size: {format_size(file_size_current)}\n"
                    f"📦 From: {file_name}\n"
                    f"📈 File {i}/{len(files)}"
                )

                # Simple upload without complex progress callbacks
                await client.send_document(
                    chat_id=message.chat.id,
                    document=file_path,
                    caption=caption,
                    file_name=base_name
                )

                uploaded_size += file_size_current
                success_count += 1

                # Send individual file completion message every 5 files
                if i % 5 == 0 or i == len(files):
                    await message.reply_text(
                        f"✅ **Upload Progress**\n\n"
                        f"📁 Last: {base_name}\n"
                        f"📈 Completed: {success_count}/{len(files)} files\n"
                        f"📊 Size: {format_size(uploaded_size)}\n"
                        f"⏱️ {'Continuing...' if i < len(files) else 'Finishing up...'}"
                    )

                await asyncio.sleep(1)  # Rate limiting

            except Exception as e:
                await message.reply_text(
                    f"❌ **Upload Failed**\n\n"
                    f"📁 File: {os.path.basename(file_path)}\n"
                    f"🔍 Error: {str(e)[:100]}...\n"
                    f"⏭️ Continuing with next file..."
                )

        # Stop upload updater
        upload_updater_task.cancel()
        upload_progress_data['message'] = None

        # Final success message
        total_time = time.time() - start_time

        await progress_msg.edit_text(
            f"🎉 **Process Complete!**\n\n"
            f"📦 Archive: {file_name}\n"
            f"✅ Uploaded: {success_count}/{len(files)} files\n"
            f"📊 Total uploaded: {format_size(uploaded_size)}\n"
            f"⏱️ Total time: {format_time(total_time)}\n\n"
            f"🧹 Cleaning up temporary files..."
        )

        # Final summary message
        await message.reply_text(
            f"🎊 **All Done!**\n\n"
            f"📦 **Original Archive:** {file_name}\n"
            f"📁 **Files Extracted:** {len(files)}\n"
            f"✅ **Successfully Uploaded:** {success_count}\n"
            f"❌ **Failed:** {len(files) - success_count}\n"
            f"📊 **Total Size:** {format_size(uploaded_size)}\n"
            f"⏱️ **Total Time:** {format_time(total_time)}\n\n"
            f"🧹 **Cleanup:** Complete\n"
            f"🚀 **Ready for next archive!**"
        )

        # Cleanup
        for part in all_parts:
            if os.path.exists(part):
                os.remove(part)
        if os.path.exists(extract_dir):
            shutil.rmtree(extract_dir, ignore_errors=True)

    except Exception as e:
        # Stop any running tasks
        if 'updater_task' in locals():
            updater_task.cancel()
        if message_id in progress_data:
            del progress_data[message_id]

        await progress_msg.edit_text(
            f"❌ **Fatal Error**\n\n"
            f"📁 File: {file_name}\n"
            f"🔍 Error: {str(e)}\n\n"
            f"Please try uploading the file again or contact support."
        )

# Run the bot
async def run_bot():
    try:
        await app.start()
        print("✅ Bot is running - All updates will show in Telegram!")
        await asyncio.Event().wait()
    except KeyboardInterrupt:
        print("\n🛑 Bot stopped")
    finally:
        await app.stop()

# Start the bot
await run_bot()

🤖 Advanced Unzip Bot Started - All updates will show in Telegram!
✅ Bot is running - All updates will show in Telegram!
